In [81]:
# установка лемматизатора pymorphy2
!pip install pymorphy2

In [82]:
# Загружаем библиотеки
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.layers import Dense, Embedding, Input, concatenate, Activation, \
MaxPooling1D, Conv1D, BatchNormalization, Dropout, Conv2DTranspose, Lambda, Conv1DTranspose
from tensorflow.keras import backend as K
from tensorflow.keras.optimizers import Adam, Adadelta
from tensorflow.keras import utils
from gensim.models import word2vec

import numpy as np
import pandas as pd
import pymorphy2
import re
import os
import time
import matplotlib.pyplot as plt
import seaborn as sns
sns.set()

In [83]:
# Подключаем диск
from google.colab import files
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [84]:
# Чтение файла текста из файла, очитска от знаков препинания
def readText(fileName):
  f = open(fileName, 'r')
  text = f.read()
  # Определяем символы для удаления
  dS = ['\n', "\t", "\ufeff", ".", "_", "-", ",", "!", "?", "–", "(", ")", "«", "»", "№", ";"]

# Каждый символ в списке символов для удаления - yдаляем, заменяя на пробел
  for ds in dS: 
    text = text.replace(ds, '')

  text = re.sub('[.]', '', text)
  text = re.sub(":", " ", text)
  text = re.sub("<", " <", text)
  text = re.sub(">", "> ", text)

# Метод split разделит текст по пробелам, при помощи метода join запишем все разделенные слова снова в строку
  text = ' '.join(text.split()) 
  text = text.lower()
  return text

# Преобразование исходного текста в список из слов в нормальной форме 
def text2Words(text):
  # Создаем экземпляр класса MorphAnalyzer
  morph = pymorphy2.MorphAnalyzer()
  # Разделяем текст на пробелы
  words = text.split(' ')
  # Переводим каждое слово в нормальную форму 
  words = [morph.parse(word)[0].normal_form for word in words]  
  return words 

In [85]:
# Путь к папке с договорами
directory = '/content/drive/My Drive/datasets/texts/Договора432/'
# Выведем названия 5-ти договоров
os.listdir(directory)[250:255] 

['dogovor-ustupki-prav-payschika.txt',
 'dogovor-zaloga-s-peredachej-predmeta-zaloga-zalogoderzhatelyu.txt',
 'dogovor-zaloga-imuschestva.txt',
 'dogovor-vozmezdnoy-ustupki-prava-trebovaniya-po-dogovoru-kuplipr (1).txt',
 'dogovor-zaloga-nedvizhimosti.txt']

In [86]:
# Список, в который запишем все наши договоры
agreements = [] 
# Проходим по всем файлам в директории договоров
for filename in os.listdir(directory): 
  # Читаем текст договора
  txt = readText(directory + filename) 
  # Если текст не пустой
  if txt != '': 
    # Преобразуем файл в одну строку и добавляем в agreements
    agreements.append(readText(directory + filename)) 

In [87]:
# Здесь будут храниться все договора в виде списка слов
words = [] 
# Засечем текущее время
curTime = time.time() 
# Проходимся по всем договорам
for i in range(len(agreements)): 
  # Преобразуем очередной договор в список слов и добавляем в words
  words.append(text2Words(agreements[i])) 
print('На преобразование ушло:', round(time.time() - curTime, 2), 'с.')

На преобразование ушло: 154.21 с.


In [88]:
# Возьмем 10 текстов для финальной проверки обученной нейронной сети
wordsToTest = words[-10:]  
# Для обученающей и проверочной выборок возьмем все тексты, кроме последних 10
words = words[:-10] 

In [89]:
print('Количество текстов в выборке для финальной проверки в конце ноутбука:', len(wordsToTest)) 
print('Количество договоров, взятых для обучающей и тестовой выборки:',len(words)) 
print('Количество слов в первом договоре:', len(words[0]))
print('Пример исходного текста:')
print(agreements[4][:62], '\n')
print('Тот же текст, представленный в виде списка из слов в начальной форме:')
print(words[4][:10])

Количество текстов в выборке для финальной проверки в конце ноутбука: 10
Количество договоров, взятых для обучающей и тестовой выборки: 412
Количество слов в первом договоре: 1083
Пример исходного текста:
договор пожизненного содержания с иждивением с предоставлением 

Тот же текст, представленный в виде списка из слов в начальной форме:
['договор', 'пожизненный', 'содержание', 'с', 'иждивение', 'с', 'предоставление', 'иной', 'жилой', 'помещение']


In [90]:
# lower=True - приводим слова к нижнему регистру
# char_level=False - просим токенайзер учитывать слова, а не отдельные символы
tokenizer = Tokenizer(lower=True, filters = '', char_level=False)
 # "Скармливаем" наши тексты, т. е. даём в обработку методу, который соберет словарь частотности
tokenizer.fit_on_texts(words)
clean_voc = {} 
#Преобразуем полученный список
for item in tokenizer.word_index.items():
# В словарь, меняя местами элементы полученного кортежа   
  clean_voc[item[0]] = item[1] 

In [91]:
print('Словарь всех слов по частоте:') 
print(clean_voc, '\n')
print('Длина словаря:', len(clean_voc))

tag = '<s1>'
print('Индекс тега', tag, ':' ,clean_voc[tag])

Словарь всех слов по частоте:
{'в': 1, 'договор': 2, 'и': 3, 'по': 4, 'с': 5, 'настоящий': 6, 'на': 7, 'сторона': 8, '<s1>': 9, '</s1>': 10, 'не': 11, 'или': 12, 'за': 13, 'он': 14, 'случай': 15, 'о': 16, '<s3>': 17, '</s3>': 18, 'право': 19, 'срок': 20, 'работа': 21, '<s4>': 22, 'от': 23, '</s4>': 24, 'они': 25, 'при': 26, 'другой': 27, 'арендатор': 28, 'быть': 29, 'день': 30, 'имущество': 31, 'лицо': 32, 'действовать': 33, 'для': 34, 'к': 35, 'если': 36, 'дальнейший': 37, 'который': 38, 'условие': 39, 'а': 40, 'обязательство': 41, 'страховой': 42, 'сумма': 43, 'порядок': 44, 'действие': 45, 'работник': 46, 'арендодатель': 47, 'свой': 48, 'указанный': 49, 'размер': 50, 'соответствие': 51, 'счёт': 52, 'именовать': 53, 'из': 54, 'мочь': 55, 'законодательство': 56, 'обязать': 57, 'страховщик': 58, 'один': 59, 'исполнение': 60, 'основание': 61, 'страхователь': 62, 'обязанность': 63, 'иной': 64, 'вправе': 65, 'до': 66, 'иметь': 67, 'также': 68, 'это': 69, 'год': 70, 'ответственность': 71, 

In [92]:
# Преобразовываем текст в последовательность индексов согласно частотному словарю
# Обучающие тесты в индексы
tok_agreem = tokenizer.texts_to_sequences(words) 

print("Взглянем на фрагмент обучающего текста:")
print("В виде оригинального текста:              ", words[4][:20])
print("Он же в виде последовательности индексов: ", tok_agreem[4][:20], '\n')

Взглянем на фрагмент обучающего текста:
В виде оригинального текста:               ['договор', 'пожизненный', 'содержание', 'с', 'иждивение', 'с', 'предоставление', 'иной', 'жилой', 'помещение', 'для', 'проживание', 'взамен', 'передаваемый', 'г', 'г', 'гражданин', 'паспорт', 'серия', 'номер']
Он же в виде последовательности индексов:  [2, 1302, 451, 5, 2599, 5, 373, 64, 303, 76, 34, 1108, 4287, 372, 81, 81, 280, 278, 250, 216] 



In [93]:
# Собираем список индексов и dummy encoded вектора
def get01XSamples(tok_agreem, tags_index):
  tags01 = [] # Список для тегов
  indexes = [] # Здесь будут лежать индексы
 
  for agreement in tok_agreem: # Проходимся по каждому договору-списку
    tag_place = [0, 0, 0, 0, 0, 0] # Создаем вектор [0,0,0,0,0,0]
    for ex in agreement: # Проходимся по каждому слову договора
        if ex in tags_index: # Смотрим, если индекс оказался нашим тегом
          place = np.argwhere(tags_index==ex) # Записываем, под каким местом лежит этот тег в своем списке
          if len(place)!=0: # Проверяем, чтобы тег действительно был
            if place[0][0]<6: # Первые шесть тегов в списке - открывающие
              tag_place[place[0][0]] = 1    # Поэтому ставим 1
            else: 
              tag_place[place[0][0] - 6] = 0  # Остальные теги закрывающие, так что меняем на ноль
        else:          
          tags01.append(tag_place.copy()) # Расширяем наш список с каждой итерацией. Получаем в конце длинный список из всех тегов в одном 
          indexes.append(ex) # Докидываем индекс слова в список индексов

  return indexes, tags01

# Получение списка слов из индексов
def reverseIndex(clean_voc, x):
  reverse_word_map = dict(map(reversed, clean_voc.items())) # Берем пары значений всего словаря и размечаем наоборот, т. е. value:key
  words = [reverse_word_map.get(letter) for letter in x] # Вытаскиваем по каждому ключу в список
  return words # Возвращаем полученный текст

In [94]:
# s1 Условия
# s2 Запреты
# s3 Стоимость, всё про цены и деньги
# s4 Всё про сроки
# s5 Неустойка
# s6 Всё про адреса и геолокации

tags_index = ['<s' + str(i) + '>' for i in range(1, 7)] # Получаем список открывающих тегов
closetags = ['</s' + str(i) + '>' for i in range(1, 7)] # Получаем список закрывающих тегов
tags_index.extend(closetags) # Объединяем все теги

tags_index = np.array([clean_voc[i] for i in tags_index]) # Получаем из словаря частотности индексы всех тегов
print('Индексы тегов:', tags_index)

Индексы тегов: [  9 326  17  22 101 251  10 334  18  24 102 242]


In [95]:
# Распознаем теги и создаем список с ними, с индексами
xData, yData = get01XSamples(tok_agreem,tags_index)
# Для создания списков с embedding-ами сначала преобразуем список индексов обратно в слова
decoded_text = reverseIndex(clean_voc, xData)

In [96]:
print('Длина xData:', len(xData))
print('Длина yData:', len(yData))

print('Посмотрим на исходный набор слов: ', words[0][50:80])
print('Посмотрим на декодированный текст:', decoded_text[50:80])
print('Посмотрим на часть из xData:     ', xData[50:80])
print('Посмотрим на часть из yData:     ', yData[50:80])

Длина xData: 383015
Длина yData: 383015
Посмотрим на исходный набор слов:  ['1', 'предмет', 'договора11', '<s1>', 'по', 'настоящий', 'договор', 'хранитель', 'обязываться', 'хранить', 'вещь', 'передать', 'он', 'поклажедатель', 'и', 'возвратить', 'этот', 'вещь', 'в', 'сохранность', '</s1>', '12', 'поклажедатель', 'передавать', 'на', 'хранение', 'по', 'настоящий', 'договор', 'следующий']
Посмотрим на декодированный текст: ['1', 'предмет', 'договора11', 'по', 'настоящий', 'договор', 'хранитель', 'обязываться', 'хранить', 'вещь', 'передать', 'он', 'поклажедатель', 'и', 'возвратить', 'этот', 'вещь', 'в', 'сохранность', '12', 'поклажедатель', 'передавать', 'на', 'хранение', 'по', 'настоящий', 'договор', 'следующий', 'вещь', 'далее']
Посмотрим на часть из xData:      [92, 98, 289, 4, 6, 2, 205, 99, 933, 308, 111, 14, 226, 3, 545, 221, 308, 1, 406, 247, 226, 282, 7, 147, 4, 6, 2, 141, 308, 302]
Посмотрим на часть из yData:      [[0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0], [1, 0,

In [97]:
# Формируем выборку из индексов
def getSetFromIndexes(wordIndexes, xLen, step): 
  xBatch = [] # Лист для фрагментов текста
  wordsLen = len(wordIndexes) # Определяем длину текста
  index = 0 # Задаем стартовый индекс
  
  while (index + xLen <= wordsLen): # Пока сумма индекса с длиной фрагмента меньше или равна числу слов в выборке
    xBatch.append(wordIndexes[index:index+xLen]) # Добавляем X в лист фрагментов текста
    index += step # Сдвигаемся на step

  return xBatch # Лист для фрагментов текста

In [98]:
xLen = 256 # Длина окна
step = 30 # Шаг 
embeddingSize = 300 # Количество измерений для векторного пространства

In [99]:
# Генерируем наборы с заданными параметрами окна
xTrain = getSetFromIndexes(decoded_text, xLen, step) # Последовательность из xLen слов
yTrain = getSetFromIndexes(yData, xLen, step) # Последовательность из xLen-тегов

In [100]:
print('Длина xTrain:', len(xTrain))
print('Длина yTrain:', len(yTrain))
print('Длина примера из xTrain:',len(xTrain[0]))
print('Длина примера из yTrain:',len(yTrain[0]), '\n')
print('Пример xTrain', xTrain[0])
print('Пример yTrain', yTrain[0], '\n')

print('Первый пример xTrain:', xTrain[0][step-5:step+5])
print('Второй пример xTrain:', xTrain[1][:10])

Длина xTrain: 12759
Длина yTrain: 12759
Длина примера из xTrain: 256
Длина примера из yTrain: 256 

Пример xTrain ['договор', 'безвозмездный', 'хранение', 'до', 'востребование', 'г', 'г', 'гражданин', 'паспорт', 'серия', 'номер', 'выдать', 'проживать', 'по', 'адрес', 'именовать', 'в', 'дальнейший', 'хранитель', 'с', 'один', 'сторона', 'и', 'гражданин', 'паспорт', 'серия', 'номер', 'выдать', 'проживать', 'по', 'адрес', 'именовать', 'в', 'дальнейший', 'поклажедатель', 'с', 'другой', 'сторона', 'именовать', 'в', 'дальнейший', 'сторона', 'заключить', 'настоящий', 'договор', 'в', 'дальнейший', 'договор', 'о', 'нижеследующий', '1', 'предмет', 'договора11', 'по', 'настоящий', 'договор', 'хранитель', 'обязываться', 'хранить', 'вещь', 'передать', 'он', 'поклажедатель', 'и', 'возвратить', 'этот', 'вещь', 'в', 'сохранность', '12', 'поклажедатель', 'передавать', 'на', 'хранение', 'по', 'настоящий', 'договор', 'следующий', 'вещь', 'далее', 'вещь13', 'при', 'передача', 'вещь', 'поклажедатель', 'на',

In [101]:
# Функция, которая смотрит на пересечение областей. Нужна для accuracy
def dice_coef(y_true, y_pred):
    return (2. * K.sum(y_true * y_pred) + 1.) / (K.sum(y_true) + K.sum(y_pred) + 1.)

In [102]:
# Создаем выборки
def getSets(model, senI, tagI):
  xVector = [] # Здесь будет лежать embedding представление каждого из индексов
  tmp = [] # Временный список
  for text in senI: # Проходимся по каждому тексту-списку
    tmp=[]
    for word in text: # Проходимся по каждому слову в тексте-списке
      tmp.append(model[word]) 

    xVector.append(tmp)

  return np.array(xVector), np.array(tagI)

In [103]:
# Передаем в word2vec списки списков слов для обучения
# size = embeddingSize - размер эмбеддинга
# window = 10 - минимальное расстояние между словами в эмбеддинге 
# min_count = 1 - игнорирование всех слов с частотой, меньше, чем 1
# workers = 10 - число потоков обучения эмбеддинга
# iter = 10 - число эпох обучения эмбеддинга

modelGENSIM = word2vec.Word2Vec(xTrain, size = embeddingSize, window = 10, min_count = 1, workers = 10, iter = 10)

In [104]:
xTrainGENSIM, yTrainGENSIM = getSets(modelGENSIM, xTrain, yTrain)
print('Размерность полученного xTrain:', xTrainGENSIM.shape)
print('Размерность полученного yTrain:', yTrainGENSIM.shape)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:8: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  


Размерность полученного xTrain: (12759, 256, 300)
Размерность полученного yTrain: (12759, 256, 6)


In [105]:
np.save('xTrainGENSIM.npy', xTrainGENSIM) #Сохраняем массивы в память колаба
np.save('yTrainGENSIM.npy', yTrainGENSIM) #Сохраняем массивы в память колаба

In [106]:
xTrainGENSIM = np.load('xTrainGENSIM.npy') #Загружаем массивы из памяти колаба
yTrainGENSIM = np.load('yTrainGENSIM.npy') #Загружаем массивы из памяти колаба

In [107]:
# Функция, выводящая точность распознавания каждой категории отдельно
def recognizeSet(tagI, pred, tags, length, value):
  total=0

  for j in range(6): # общее количество тегов
    correct=0
    for i in range(len(tagI)): # проходимся по каждому списку списка тегов
      for k in range(length): # проходимся по каждому тегу
        if tagI[i][k][j]==(pred[i][k][j]>value).astype(int): # если соответствующие индексы совпадают, значит сеть распознала верно
          correct+=1 
    print("Сеть распознала категорию '{}' на {}%".format(tags[j],round(100*correct/(len(tagI)*length), 2)))
    total += 100 * correct / (len(tagI)*length)
  print("Cредняя точность {}%".format(total/6))

In [108]:
# Преобразовываем текст в последовательность индексов согласно частотному словарю
tok_agreemTest = tokenizer.texts_to_sequences(wordsToTest) # Обучающие тесты в индексы

print("Взглянем на фрагмент тестового текста:")
print("В виде оригинального текста:              ", wordsToTest[4][:20])
print("Он же в виде последовательности индексов: ", tok_agreemTest[4][:20], '\n')

Взглянем на фрагмент тестового текста:
В виде оригинального текста:               ['трудовой', 'контракт', 'с', 'руководитель', 'предприятие', 'о', 'предпринимательский', 'деятельность', 'г', 'г', 'в', 'лицо', 'действовать', 'на', 'основание', 'именовать', 'в', 'дальнейший', 'собственник', 'с']
Он же в виде последовательности индексов:  [73, 355, 5, 838, 420, 16, 1340, 156, 81, 81, 1, 32, 33, 7, 61, 53, 1, 37, 437, 5] 



In [109]:
xDataTest, yDataTest = get01XSamples(tok_agreemTest,tags_index) # Распознаем теги и создаем список с ними, с индексами
decoded_text = reverseIndex(clean_voc, xDataTest) # Для создания списков с embedding-ами сначала преобразуем список индексов обратно в слова

In [110]:
print('Длина xDataTest:', len(xDataTest))
print('Длина yDataTest:', len(yDataTest))

print('Посмотрим на исходный набор слов: ', wordsToTest[0][50:80])
print('Посмотрим на декодированный текст:', decoded_text[50:80])
print('Посмотрим на часть из xDataTest:     ', xDataTest[50:80])
print('Посмотрим на часть из yDataTest:     ', yDataTest[50:80])

Длина xDataTest: 10494
Длина yDataTest: 10494
Посмотрим на исходный набор слов:  ['дальнейший', 'директор', 'с', 'другой', 'стороны12', 'настоящий', 'контракт', 'регулировать', 'трудовой', 'и', 'иной', 'отношение', 'между', 'директор', 'и', 'товарищество', 'определять', 'право', 'обязанность', 'и', 'ответственность', 'директор', 'перед', 'товариществомустанавливать', 'размер', 'оплата', 'он', 'труд', 'другой', 'вознаграждение']
Посмотрим на декодированный текст: ['дальнейший', 'директор', 'с', 'другой', 'настоящий', 'контракт', 'регулировать', 'трудовой', 'и', 'иной', 'отношение', 'между', 'директор', 'и', 'товарищество', 'определять', 'право', 'обязанность', 'и', 'ответственность', 'директор', 'перед', 'размер', 'оплата', 'он', 'труд', 'другой', 'вознаграждение', 'и', 'социальнобытовый']
Посмотрим на часть из xDataTest:      [37, 323, 5, 27, 6, 355, 1130, 73, 3, 64, 243, 173, 323, 3, 1371, 708, 19, 63, 3, 71, 323, 307, 50, 163, 14, 246, 27, 362, 3, 2777]
Посмотрим на часть из yDataTes

In [111]:
xLen = 256 # Длина окна
step = 30 # Шаг 
embeddingSize = 300 # Количество измерений для векторного пространства

# Генерируем наборы с заданными параметрами окна
xTest = getSetFromIndexes(decoded_text, xLen, step) # Последовательность из xLen слов
yTest = getSetFromIndexes(yDataTest, xLen, step) # Последовательность из xLen-тегов

In [112]:
print('Длина xTest:', len(xTest))
print('Длина yTest:', len(yTest))
print('Длина примера из xTest:',len(xTest[0]))
print('Длина примера из yTrain:',len(yTest[0]), '\n')
print('Пример xTest', xTest[0])
print('Пример yTest', yTest[0], '\n')

print('Первый пример xTest:', xTest[0][step-5:step+5])
print('Второй пример xTest:', xTest[1][:10])

Длина xTest: 342
Длина yTest: 342
Длина примера из xTest: 256
Длина примера из yTrain: 256 

Пример xTest ['трудовой', 'контракт', 'с', 'генеральный', 'директор', 'товарищество', 'с', 'ограниченный', 'ответственность', 'г', 'г1', 'общий', 'положения11', 'настоящий', 'контракт', 'заключить', 'между', 'товарищество', 'с', 'ограниченный', 'ответственность', 'именовать', 'в', 'дальнейший', 'товарищество', 'в', 'лицо', 'председатель', 'совет', 'участник', 'товарищество', 'действовать', 'на', 'основание', 'решение', 'общий', 'собрание', 'товарищество', 'протокол', 'от', 'год', 'с', 'один', 'сторона', 'и', 'гражданин', 'российский', 'федерация', 'именовать', 'в', 'дальнейший', 'директор', 'с', 'другой', 'настоящий', 'контракт', 'регулировать', 'трудовой', 'и', 'иной', 'отношение', 'между', 'директор', 'и', 'товарищество', 'определять', 'право', 'обязанность', 'и', 'ответственность', 'директор', 'перед', 'размер', 'оплата', 'он', 'труд', 'другой', 'вознаграждение', 'и', 'социальнобытовый', 'ос

In [113]:
# Передаем в word2vec списки списков слов для обучения
# size = embeddingSize - размер эмбеддинга
# window = 10 - расстояние между текущим и прогнозируемым словом в предложении
# min_count = 1 - игнорирование всех слов с частотой, меньше, чем 1
# workers = 10 - число потоков обучения эмбеддинга
# iter = 10 - число эпох обучения эмбеддинга

modelGENSIM = word2vec.Word2Vec(xTest, size = embeddingSize, window = 10, min_count = 1, workers = 10, iter = 10)

In [114]:
xTestGENSIM, yTestGENSIM = getSets(modelGENSIM, xTest, yTest)
print('Размерность полученного xTrain:', xTestGENSIM.shape)
print('Размерность полученного yTrain:', yTestGENSIM.shape)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:8: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  


Размерность полученного xTrain: (342, 256, 300)
Размерность полученного yTrain: (342, 256, 6)


In [115]:
tags = ['S1', 'S2', 'S3', 'S4', 'S5', 'S6']
# s1 Условия
# s2 Запреты
# s3 Стоимость, всё про цены и деньги
# s4 Всё про сроки
# s5 Неустойка
# s6 Всё про адреса и геолокации

In [116]:
#Функция для создания сети UNET подобной сети, без конкатеции слоев
def my_net(k = 1, k_size = 3, num_classes = 6, input_shape= (30, 300)):
        
    img_input = Input(input_shape) 

    #1
    x = Conv1D(64 * k , k_size, padding='same')(img_input) 
    x = BatchNormalization()(x) 
    x = Activation('relu')(x)

    x = Conv1D(64 * k , k_size, padding='same')(x)  
    x = BatchNormalization()(x)     
    x = Activation('relu')(x) 

    x = MaxPooling1D()(x)

    #2
    x = Conv1D(128 * k , k_size, padding='same')(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)  

    x = Conv1D(128 * k , k_size, padding='same')(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)

    x = MaxPooling1D()(x)

    #3
    x = Conv1D(256 * k , k_size, padding='same')(x)
    x = BatchNormalization()(x)               
    x = Activation('relu')(x)                     

    x = Conv1D(256 * k , k_size, padding='same')(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)

    x = Conv1D(256 * k , k_size, padding='same')(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    
    x = MaxPooling1D()(x)

    #4
    x = Conv1D(512 * k , k_size, padding='same')(x)
    x = BatchNormalization()(x) 
    x = Activation('relu')(x)

    x = Conv1D(512 * k , k_size, padding='same')(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)

    x = Conv1D(512 * k , k_size, padding='same')(x)
    x = BatchNormalization()(x)      
    x = Activation('relu')(x) 

    # UP 2
    x = Conv1DTranspose(256, kernel_size=2, strides=2, padding='same')(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x) 

    x = Conv1D(256 * k , k_size, padding='same')(x) 
    x = BatchNormalization()(x) 
    x = Activation('relu')(x)

    x = Conv1D(256 * k , k_size, padding='same')(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    
    # UP 3
    x = Conv1DTranspose(128, kernel_size=2, strides=2, padding='same')(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x) 

    x = Conv1D(128 * k , k_size, padding='same')(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)

    x = Conv1D(128 * k , k_size, padding='same')(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)

    # UP 4
    x = Conv1DTranspose(64, kernel_size=2, strides=2, padding='same')(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)

    x = Conv1D(64 * k , k_size, padding='same')(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)

    x = Conv1D(64 * k , k_size, padding='same')(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)

    x = Conv1D(num_classes, k_size, activation='sigmoid', padding='same')(x)

    model = Model(img_input, x) 

    model = Model(img_input, x)
    model.compile(optimizer=Adam(0.0025), 
                  loss='categorical_crossentropy',
                  metrics=[dice_coef])
  
    return model

In [117]:
model_my = my_net(input_shape=(xLen, embeddingSize))

In [118]:
history_my = model_my.fit(xTrainGENSIM, yTrainGENSIM, epochs=20, batch_size=64, validation_split = 0.2)

Epoch 1/20
160/160 [==============================] - 16s 38ms/step - loss: 1.0703 - dice_coef: 0.3521 - val_loss: 156.4794 - val_dice_coef: 0.2337
Epoch 2/20
160/160 [==============================] - 5s 33ms/step - loss: 1.2901 - dice_coef: 0.3669 - val_loss: 27.5541 - val_dice_coef: 0.2336
Epoch 3/20
160/160 [==============================] - 5s 32ms/step - loss: 1.9868 - dice_coef: 0.3290 - val_loss: 11.0482 - val_dice_coef: 0.2549
Epoch 4/20
160/160 [==============================] - 5s 32ms/step - loss: 3.0691 - dice_coef: 0.3324 - val_loss: 10.5321 - val_dice_coef: 0.2669
Epoch 5/20
160/160 [==============================] - 5s 32ms/step - loss: 4.3592 - dice_coef: 0.3365 - val_loss: 9.3240 - val_dice_coef: 0.2670
Epoch 6/20
160/160 [==============================] - 5s 32ms/step - loss: 6.0449 - dice_coef: 0.3382 - val_loss: 4.3754 - val_dice_coef: 0.2688
Epoch 7/20
160/160 [==============================] - 5s 32ms/step - loss: 6.9378 - dice_coef: 0.3402 - val_loss: 12.1250 - 

In [119]:
pred_my = model_my.predict(xTestGENSIM) # сделаем предсказание

In [120]:
recognizeSet(yTestGENSIM, pred_my, tags, xLen, 0.999)

Сеть распознала категорию 'S1' на 63.44%
Сеть распознала категорию 'S2' на 97.61%
Сеть распознала категорию 'S3' на 84.45%
Сеть распознала категорию 'S4' на 86.71%
Сеть распознала категорию 'S5' на 92.06%
Сеть распознала категорию 'S6' на 99.42%
Cредняя точность 87.28241502192982%


In [121]:
model_my2 = my_net(k=0.5, input_shape=(xLen, embeddingSize)) #уменьшим количество нейронов в 2 раза от исходной
history_my2 = model_my2.fit(xTrainGENSIM, yTrainGENSIM, epochs=20, batch_size=64, validation_split = 0.2)
pred_my2 = model_my2.predict(xTestGENSIM)
recognizeSet(yTestGENSIM, pred_my2, tags, xLen, 0.999)

Epoch 1/20
160/160 [==============================] - 10s 34ms/step - loss: 1.0429 - dice_coef: 0.3296 - val_loss: 132.3815 - val_dice_coef: 0.2592
Epoch 2/20
160/160 [==============================] - 4s 28ms/step - loss: 1.3797 - dice_coef: 0.3590 - val_loss: 42.0301 - val_dice_coef: 0.2432
Epoch 3/20
160/160 [==============================] - 4s 28ms/step - loss: 1.7187 - dice_coef: 0.3360 - val_loss: 11.9526 - val_dice_coef: 0.2568
Epoch 4/20
160/160 [==============================] - 4s 28ms/step - loss: 2.4531 - dice_coef: 0.3360 - val_loss: 5.8294 - val_dice_coef: 0.2652
Epoch 5/20
160/160 [==============================] - 4s 28ms/step - loss: 3.1455 - dice_coef: 0.3358 - val_loss: 3.9364 - val_dice_coef: 0.2628
Epoch 6/20
160/160 [==============================] - 4s 28ms/step - loss: 4.0187 - dice_coef: 0.3374 - val_loss: 2.5907 - val_dice_coef: 0.2711
Epoch 7/20
160/160 [==============================] - 4s 28ms/step - loss: 5.5132 - dice_coef: 0.3398 - val_loss: 4.2324 - va

In [122]:
model_my3 = my_net(k=2, input_shape=(xLen, embeddingSize)) #увеличим количество нейронов в 2 раза от исходной
history_my3 = model_my3.fit(xTrainGENSIM, yTrainGENSIM, epochs=20, batch_size=64, validation_split = 0.2)
pred_my3 = model_my3.predict(xTestGENSIM)
recognizeSet(yTestGENSIM, pred_my3, tags, xLen, 0.999)

Epoch 1/20
160/160 [==============================] - 14s 57ms/step - loss: 1.0752 - dice_coef: 0.3577 - val_loss: 236.7021 - val_dice_coef: 0.2342
Epoch 2/20
160/160 [==============================] - 8s 52ms/step - loss: 1.9453 - dice_coef: 0.3295 - val_loss: 42.7339 - val_dice_coef: 0.2320
Epoch 3/20
160/160 [==============================] - 8s 52ms/step - loss: 3.1736 - dice_coef: 0.3297 - val_loss: 20.8089 - val_dice_coef: 0.2642
Epoch 4/20
160/160 [==============================] - 8s 52ms/step - loss: 4.9918 - dice_coef: 0.3345 - val_loss: 11.3967 - val_dice_coef: 0.2711
Epoch 5/20
160/160 [==============================] - 8s 52ms/step - loss: 7.5603 - dice_coef: 0.3371 - val_loss: 9.3317 - val_dice_coef: 0.2677
Epoch 6/20
160/160 [==============================] - 8s 52ms/step - loss: 9.5955 - dice_coef: 0.3389 - val_loss: 6.2432 - val_dice_coef: 0.2570
Epoch 7/20
160/160 [==============================] - 8s 52ms/step - loss: 11.6223 - dice_coef: 0.3407 - val_loss: 8.8149 - 

In [123]:
model_my4 = my_net(k=0.25, input_shape=(xLen, embeddingSize)) #уменьшим количество нейронов в 4 раза от исходной
history_my4 = model_my4.fit(xTrainGENSIM, yTrainGENSIM, epochs=20, batch_size=64, validation_split = 0.2)
pred_my4 = model_my4.predict(xTestGENSIM)
recognizeSet(yTestGENSIM, pred_my4, tags, xLen, 0.999)

Epoch 1/20
160/160 [==============================] - 10s 35ms/step - loss: 1.0917 - dice_coef: 0.2958 - val_loss: 6.7832 - val_dice_coef: 0.3182
Epoch 2/20
160/160 [==============================] - 5s 29ms/step - loss: 1.1319 - dice_coef: 0.3778 - val_loss: 61.5944 - val_dice_coef: 0.2588
Epoch 3/20
160/160 [==============================] - 4s 28ms/step - loss: 1.4264 - dice_coef: 0.3534 - val_loss: 3.6021 - val_dice_coef: 0.2621
Epoch 4/20
160/160 [==============================] - 4s 28ms/step - loss: 1.9548 - dice_coef: 0.3353 - val_loss: 2.6873 - val_dice_coef: 0.2652
Epoch 5/20
160/160 [==============================] - 4s 28ms/step - loss: 2.3720 - dice_coef: 0.3337 - val_loss: 4.0392 - val_dice_coef: 0.2588
Epoch 6/20
160/160 [==============================] - 4s 28ms/step - loss: 3.0115 - dice_coef: 0.3352 - val_loss: 4.5348 - val_dice_coef: 0.2654
Epoch 7/20
160/160 [==============================] - 4s 28ms/step - loss: 3.7437 - dice_coef: 0.3375 - val_loss: 4.1826 - val_d

In [124]:
model_my5 = my_net(k=0.5, k_size=2, input_shape=(xLen, embeddingSize))
history_my5 = model_my5.fit(xTrainGENSIM, yTrainGENSIM, epochs=20, batch_size=64, validation_split = 0.2)
pred_my5 = model_my5.predict(xTestGENSIM)
recognizeSet(yTestGENSIM, pred_my5, tags, xLen, 0.999)

Epoch 1/20
160/160 [==============================] - 11s 36ms/step - loss: 1.0511 - dice_coef: 0.2868 - val_loss: 3.9736 - val_dice_coef: 0.0463
Epoch 2/20
160/160 [==============================] - 5s 29ms/step - loss: 0.9792 - dice_coef: 0.4022 - val_loss: 3.3968 - val_dice_coef: 0.1826
Epoch 3/20
160/160 [==============================] - 5s 28ms/step - loss: 1.1647 - dice_coef: 0.3514 - val_loss: 5.5772 - val_dice_coef: 0.2609
Epoch 4/20
160/160 [==============================] - 5s 29ms/step - loss: 1.3536 - dice_coef: 0.3328 - val_loss: 1.7277 - val_dice_coef: 0.2556
Epoch 5/20
160/160 [==============================] - 5s 30ms/step - loss: 1.6056 - dice_coef: 0.3280 - val_loss: 2.2603 - val_dice_coef: 0.2634
Epoch 6/20
160/160 [==============================] - 5s 29ms/step - loss: 1.9185 - dice_coef: 0.3324 - val_loss: 3.8071 - val_dice_coef: 0.2650
Epoch 7/20
160/160 [==============================] - 5s 29ms/step - loss: 2.2663 - dice_coef: 0.3361 - val_loss: 3.1383 - val_di

In [125]:
model_my6 = my_net(k=0.5, k_size=4, input_shape=(xLen, embeddingSize))
history_my6 = model_my6.fit(xTrainGENSIM, yTrainGENSIM, epochs=20, batch_size=64, validation_split = 0.2)
pred_my6 = model_my6.predict(xTestGENSIM)
recognizeSet(yTestGENSIM, pred_my5, tags, xLen, 0.999)

Epoch 1/20
160/160 [==============================] - 10s 35ms/step - loss: 1.0116 - dice_coef: 0.3242 - val_loss: 10.1808 - val_dice_coef: 0.1677
Epoch 2/20
160/160 [==============================] - 5s 28ms/step - loss: 0.9645 - dice_coef: 0.4065 - val_loss: 5.1166 - val_dice_coef: 0.2571
Epoch 3/20
160/160 [==============================] - 5s 29ms/step - loss: 1.2203 - dice_coef: 0.3372 - val_loss: 8.4407 - val_dice_coef: 0.2656
Epoch 4/20
160/160 [==============================] - 5s 28ms/step - loss: 1.3709 - dice_coef: 0.3293 - val_loss: 1.7983 - val_dice_coef: 0.2647
Epoch 5/20
160/160 [==============================] - 5s 28ms/step - loss: 1.6453 - dice_coef: 0.3312 - val_loss: 3.3753 - val_dice_coef: 0.2651
Epoch 6/20
160/160 [==============================] - 5s 28ms/step - loss: 1.9620 - dice_coef: 0.3356 - val_loss: 1.6834 - val_dice_coef: 0.2552
Epoch 7/20
160/160 [==============================] - 5s 29ms/step - loss: 2.1961 - dice_coef: 0.3391 - val_loss: 2.4427 - val_d